In [1]:
# 用前须知

## xtdata提供和MiniQmt的交互接口，本质是和MiniQmt建立连接，由MiniQmt处理行情数据请求，再把结果回传返回到python层。使用的行情服务器以及能获取到的行情数据和MiniQmt是一致的，要检查数据或者切换连接时直接操作MiniQmt即可。

## 对于数据获取接口，使用时需要先确保MiniQmt已有所需要的数据，如果不足可以通过补充数据接口补充，再调用数据获取接口获取。

## 对于订阅接口，直接设置数据回调，数据到来时会由回调返回。订阅接收到的数据一般会保存下来，同种数据不需要再单独补充。

# 代码讲解

# 从本地python导入xtquant库，如果出现报错则说明安装失败
from xtquant import xtdata
import pandas as pd
import time
import copy

xtquant文档地址：http://dict.thinktrader.net/nativeApi/start_now.html


In [4]:

# 设定一个标的列表
code_list = ["ru00.SF"]
# 判断当前主力合约
main_contract = [xtdata.get_main_contract(code) for code in code_list]
print(f"当前主力合约: {main_contract}")

# 设定获取数据的周期
period = "tick"


***** xtdata连接成功 *****
服务信息: {'tag': 'sp3', 'version': '1.0'}
服务地址: 127.0.0.1:58610
数据路径: C:\国金证券QMT交易端\bin.x64/../userdata_mini/datadir
设置xtdata.enable_hello = False可隐藏此消息

当前主力合约: ['ru2505.SF']


In [21]:

# 下载标的行情数据
if 1:
    ## 为了方便用户进行数据管理，xtquant的大部分历史数据都是以压缩形式存储在本地的
    ## 比如行情数据，需要通过download_history_data下载，财务数据需要通过
    ## 所以在取历史数据之前，我们需要调用数据下载接口，将数据下载到本地
    for contract in main_contract:
        xtdata.download_history_data(contract, period=period, incrementally=True) # 增量下载行情数据（开高低收,等等）到本地
    
    # xtdata.download_financial_data(code_list) # 下载财务数据到本地
    # xtdata.download_sector_data() # 下载板块数据到本地
    # # 更多数据的下载方式可以通过数据字典查询


In [29]:
# 读取本地历史行情数据
history_data = xtdata.get_market_data_ex([], main_contract, period=period, start_time='20230613', count=-1)
history_data[main_contract[0]]

,time,lastPrice,open,high,low,lastClose,amount,volume,pvolume,tickvol,stockStatus,openInt,lastSettlementPrice,askPrice,bidPrice,askVol,bidVol,settlementPrice,transactionNum,pe
20241209085900,1733705940500,18845.0,18970.0,19040.0,18820.0,19030.0,4.720146e+10,249473,0,4207895249583022124,0,206970,18900.0,"[18850.0, 18855.0, 18860.0, 18865.0, 18870.0]","[18840.0, 18835.0, 18830.0, 18825.0, 18820.0]","[94, 0, 0, 0, 0]","[1, 0, 0, 0, 0]",0.0,0,0.0
20241209090000,1733706000500,18850.0,18970.0,19040.0,18820.0,19030.0,4.724124e+10,249684,0,4207895249583022124,0,207039,18900.0,"[18855.0, 18860.0, 18865.0, 18870.0, 18875.0]","[18845.0, 18840.0, 18835.0, 18830.0, 18825.0]","[254, 0, 0, 0, 0]","[59, 0, 0, 0, 0]",0.0,0,0.0
20241209090001,1733706001000,18845.0,18970.0,19040.0,18820.0,19030.0,4.726612e+10,249816,0,0,0,207024,18900.0,"[18850.0, 18855.0, 18860.0, 18865.0, 18870.0]","[18840.0, 18835.0, 18830.0, 18825.0, 18820.0]","[381, 0, 0, 0, 0]","[2, 0, 0, 0, 0]",0.0,0,0.0
20241209090001,1733706001500,18845.0,18970.0,19040.0,18820.0,19030.0,4.727611e+10,249869,0,7310027618099670065,0,207021,18900.0,"[18850.0, 18855.0, 18860.0, 18865.0, 18870.0]","[18840.0, 18835.0, 18830.0, 18825.0, 18820.0]","[221, 0, 0, 0, 0]","[27, 0, 0, 0, 0]",0.0,0,0.0
20241209090002,1733706002000,18840.0,18970.0,19040.0,18820.0,19030.0,4.729080e+10,249947,0,0,0,207036,18900.0,"[18845.0, 18850.0, 18855.0, 18860.0, 18865.0]","[18835.0, 18830.0, 18825.0, 18820.0, 18815.0]","[16, 0, 0, 0, 0]","[18, 0, 0, 0, 0]",0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20250109225515,1736434515000,16960.0,16920.0,17030.0,16900.0,16910.0,2.471710e+10,145640,0,0,0,198463,16825.0,"[16960.0, 0.0, 0.0, 0.0, 0.0]","[16960.0, 0.0, 0.0, 0.0, 0.0]","[79, 0, 0, 0, 0]","[145, 0, 0, 0, 0]",0.0,0,0.0
20250109225515,1736434515500,16965.0,16920.0,17030.0,16900.0,16910.0,2.471761e+10,145643,0,0,0,198463,16825.0,"[16965.0, 0.0, 0.0, 0.0, 0.0]","[16965.0, 0.0, 0.0, 0.0, 0.0]","[79, 0, 0, 0, 0]","[143, 0, 0, 0, 0]",0.0,0,0.0
20250109225516,1736434516000,16965.0,16920.0,17030.0,16900.0,16910.0,2.471761e+10,145643,0,0,0,198463,16825.0,"[16965.0, 0.0, 0.0, 0.0, 0.0]","[16965.0, 0.0, 0.0, 0.0, 0.0]","[79, 0, 0, 0, 0]","[144, 0, 0, 0, 0]",0.0,0,0.0
20250109225516,1736434516500,16965.0,16920.0,17030.0,16900.0,16910.0,2.471761e+10,145643,0,0,0,198463,16825.0,"[16965.0, 0.0, 0.0, 0.0, 0.0]","[16965.0, 0.0, 0.0, 0.0, 0.0]","[80, 0, 0, 0, 0]","[148, 0, 0, 0, 0]",0.0,0,0.0


In [34]:
df = copy.deepcopy(history_data[main_contract[0]])
df['askVol'] = df['askVol'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['bidVol'] = df['bidVol'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['askPrice'] = df['askPrice'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['bidPrice'] = df['bidPrice'].apply(lambda x: x[0] if isinstance(x, list) else x)
df['open'] = df['open'] - df['lastPrice']
# df['lastClose'] = df['lastClose'] - df['lastPrice']
df = df.drop(columns=['askPrice', 'bidPrice', 'askVol', 'bidVol', 'time', 'high', 'low', 'pvolume', 'stockStatus', 'lastSettlementPrice', 'settlementPrice', 'transactionNum', 'pe', 'lastClose', 'amount'])
df

,lastPrice,open,volume,tickvol,openInt
20241209085900,18845.0,125.0,249473,4207895249583022124,206970
20241209090000,18850.0,120.0,249684,4207895249583022124,207039
20241209090001,18845.0,125.0,249816,0,207024
20241209090001,18845.0,125.0,249869,7310027618099670065,207021
20241209090002,18840.0,130.0,249947,0,207036
...,...,...,...,...,...
20250109225515,16960.0,-40.0,145640,0,198463
20250109225515,16965.0,-45.0,145643,0,198463
20250109225516,16965.0,-45.0,145643,0,198463
20250109225516,16965.0,-45.0,145643,0,198463


In [35]:
import matplotlib.pyplot as plt

import pmdarima as pm

# 提取 lastPrice 列
last_price_series = df['lastPrice']

# 拆分训练集和测试集
train_size = int(len(last_price_series) * 0.8)
train, test = last_price_series[:train_size], last_price_series[train_size:]

# 使用自动ARIMA来寻找最佳参数
auto_model = pm.auto_arima(train, seasonal=False, stepwise=True, trace=True)

# 打印最佳参数
print(f"最佳ARIMA参数: {auto_model.order}")

# 使用最佳参数重新建立ARIMA模型
model = ARIMA(train, order=auto_model.order)
model_fit = model.fit()

# 预测
forecast = model_fit.forecast(steps=len(test))

# 绘制结果
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast, label='Forecast')
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'statsmodels'